# Operator Overloading
- Let class intercept normal Pytonh operations
- Make class instances act more like built-in types

## Performance
Don't expect speed adavtange  
In fact, it might be slower. (Might due to the overhead of a function call)  

In [10]:
import timeit

min(timeit.repeat("L = list(range(100)); x = L.__len__()", number=10000, repeat=3))

0.044870924118527

In [11]:
min(timeit.repeat("L = list(range(100)); x = len(L)", number=10000, repeat=3))

0.03998297356558567

## Command Used Operator
```
__init__
__repr, __str__
__call__
__getattr__
__setattr__ 
__getitem__
__setitem__                                
__len__
__bool__
__lt__,__gt__,__le__,__ge__,__eq__,__ne__
__iter__, __next__
__contains__
__index__
```

We'll dig into detail later

## Indexing and Slicing: `__getitem__`, `__setitem__`

### `__getitem__`
It's called for instance-indexing operation. (e.g. X[i])

In [16]:
class Indexer(object):
    data = [5, 6, 7, 8, 9]
    def __getitem__(self, index):
        print("getitem: ", index)
        return self.data[index]
    
x = Indexer()
x[2]

getitem:  2


7

In [17]:
x[2:4]

getitem:  slice(2, 4, None)


[7, 8]

Handle the slice object (slice objects have attribute **start**, **stop** and **step**)

In [21]:
class Indexer(object):
    data = [5, 6, 7, 8, 9]
    def __getitem__(self, index):
        if isinstance(index, int):
            print('indexing', index)
        else:
            print('slicing', index.start, index.stop, index.step)
        return self.data[index]
    
x = Indexer()
x[2:4]

slicing 2 4 None


[7, 8]

#### Code one, get a bunch free
In absence of more-specific methods, `__getitem__` may be used in the following cases.

- iteration
- **in**
- list comprehensions
- **map**
- list and tuple assignment
- type constructors

In [36]:
class StepIndexer(object):
    def __getitem__(self, i):
        return self.data[i]

x = StepIndexer()
x.data = '1234'

In [37]:
# iteration

for i in x:
    print(i)

1
2
3
4


In [38]:
# in

'1' in x

True

In [39]:
# list comprehensions

[i for i in x]

['1', '2', '3', '4']

In [40]:
# map

list(map(lambda x: int(x) * 2, x))

[2, 4, 6, 8]

In [42]:
# tuple assignment

(a, b, c, d) = x
a, b, d

('1', '2', '4')

In [43]:
# type constructor

list(x)

['1', '2', '3', '4']

### `__setitem__`

In [24]:
class IndexSeter(object):
    data = [1, 2, 3, 4]
    def __setitem__(self, index, value):
        self.data[index] = value

x = IndexSeter()
x[2] = 5
x.data

[1, 2, 5, 4]

### Slicing and Indexing in Python2
In Python 2 only , there are also `__getslice__` and `__setslice__`  
They're removed in Python3, thus, even in Python2 `__getitem__` and `__setitem__` should be used

### Python3's `__index__` is not indexing
It returns an iteger value for an instance when needed and is used by built-ins that convert to digit strins

In [25]:
class C(object):
    def __index__(self):
        return 255
x = C()
hex(x)

'0xff'

## Iterable Objects: `__iter__`, `__next__`
Iteration conect try **`__iter__`** first than **`__getitem))`**  
Generally **`__iter__`** is prefered, it supports general iteration context better than **`__getitem__`**  

When **`__iter__`** is invoked, it's expected to return an iterator object.  
If it's provided, Python thne repeated ly calls this iterator object's **`__next__`** to produce items until a **StopIteration** exception

It's designed for iteration, not random indexing.  
Thus, if indexing is needed, **`__getitem__`** should still be used.

### Single Traversal

In [47]:
class Squares(object):
    def __init__(self, start, stop):
        self.value = start - 1 
        self.stop = stop
    def __iter__(self):
        return self
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2
    
x = Squares(1, 5)
I1 = iter(x)
I2 = iter(x)

print(next(I1))
print(next(I2))

1
4


### Multiple Iterators on One Object

- When **yield** is used, it return s a new generator object and create **`__iter__`** and **`__next__`**
    - It's still true even if the generator function with a **tield** happens to be a method named **`__iter__`**

In [56]:
class Squares(object):
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        for value in range(self.start, self.stop + 1):
            yield value ** 2

x = Squares(1, 5)
I1 = iter(x)
I2 = iter(x)

print(next(I1))
print(next(I1))
print(next(I1))

print(next(I2))

1
4
9
1


- Supplemental class that stores iterator state    


In [58]:
class Squares(object):
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        return SquaresIter(self.start, self.stop)
    
class SquaresIter(object):
    def __init__(self, start, stop):
        self.value = start - 1
        self.stop = stop
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2
    
x = Squares(1, 5)
I1 = iter(x)
I2 = iter(x)

print(next(I1))
print(next(I1))
print(next(I1))

print(next(I2))

1
4
9
1


## Membership: `__contains__`, `__iter__`, `__getitem__`
Classes can implement the **in** membership operator as an iteration, using **`__iter__`** or **`__getitem__`**  
**`__contains__`**should define membership as applying to keys for a mapping and as a search for sequences  

In [60]:
class C(object):
    data = [1, 2, 3, 4]
    def __contains__(self, x):
        return x in self.data
    
x = C()
1 in x     

True

## Attribute Access: `__getattr__` and `__setattr__`

## String Representation: `__repr__`, `__str__`

## Binary Operation (e.g. __add__, __radd__, __iadd__)

## Call Expressions: `__call__`

## Comparisons (e.g. `__lt__`, `__gt__`)

## Boolean Tests: `__bool__`, `__len__`
Boolean test try **`__bool__`** first then `__len__`

## Object Destruction: `__del__`